In [1]:
import os
import time
from os.path import join, exists
from os import listdir, makedirs
from datetime import datetime
from google import genai
from google.genai import types
from openai import OpenAI
from openai import AsyncOpenAI
import requests
import json
from pydantic import BaseModel, Field
from crawl4ai import *
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.gemini import GeminiModel
from dataclasses import dataclass
from rich import print as rprint
from rich.console import Console
from rich.markdown import Markdown
import asyncio
import nest_asyncio 
# Add this line to allow nested event loops
nest_asyncio.apply()

from WebSearchAgent import *
from agent_tools import *
from agent_utils import *

config = Config()

In [2]:
model = GeminiModel(config.FLASH2_MODEL)

class Deps(BaseModel):
    #questions: dict[int, str] = Field(description="A list of questions that need to be answered.")
    @property
    def current_datetime_str(self) -> str:
        """Get the current date and time."""
        now = self.get_datetime()
        return str(now)

    def get_datetime(self):
        return datetime.now()

questions = [
    "Explain the concept of 'artificial intelligence' in a way that a 10-year-old could understand.", 
    "Compare and contrast the philosophies of Plato and Aristotle, highlighting their key differences and similarities in their views on ethics and knowledge.", 
    "Write a short poem about the feeling of walking through a forest in autumn.",
    "If someone is planning a trip to Italy and enjoys art and history, what are three cities you would recommend they visit and why?",
    "Summarize the main arguments for and against universal basic income."]

dict_questions = {k: question for (k, question) in enumerate(questions)}

deps = Deps()

agent = Agent(
    model,
    deps_type=Deps,  
    #result_type=pass,
    system_prompt="""
    You are a helpful assistant.
    """)

@agent.system_prompt  
def add_the_date(ctx: RunContext[str]) -> str:
    return f"The user's name is {ctx.deps.current_datetime_str}."

In [ ]:
for k in range(3):
    result = await agent.run('What date and time is it?', deps=deps)
    print(result.data)
    time.sleep(1)
